In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [830 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,756 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,077 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launch

In [2]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
data = spark.read.csv("/content/titanic_train.csv", header=True, sep=',', inferSchema=True)
data.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [4]:
data= data.drop(*['PassengerId', 'Name', 'Ticket'])

data.show(5)

+--------+------+------+----+-----+-----+-------+-----+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Cabin|Embarked|
+--------+------+------+----+-----+-----+-------+-----+--------+
|       0|     3|  male|22.0|    1|    0|   7.25| NULL|       S|
|       1|     1|female|38.0|    1|    0|71.2833|  C85|       C|
|       1|     3|female|26.0|    0|    0|  7.925| NULL|       S|
|       1|     1|female|35.0|    1|    0|   53.1| C123|       S|
|       0|     3|  male|35.0|    0|    0|   8.05| NULL|       S|
+--------+------+------+----+-----+-----+-------+-----+--------+
only showing top 5 rows



In [5]:
null_counts = data.agg(*[
    F.sum(F.col(c).isNull().cast("int")).alias(c)
    for c in data.columns
])

null_counts.show()

+--------+------+---+---+-----+-----+----+-----+--------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|Cabin|Embarked|
+--------+------+---+---+-----+-----+----+-----+--------+
|       0|     0|  0|177|    0|    0|   0|  687|       2|
+--------+------+---+---+-----+-----+----+-----+--------+



In [6]:
data= data.drop(*['Cabin'])

data.show(5)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 5 rows



In [7]:
data = data.na.drop()

data.show(5)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 5 rows



In [8]:
data.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



In [9]:
sex_indexer = StringIndexer(inputCol='Sex', outputCol='Sex_i')
embarked_indexer = StringIndexer(inputCol='Embarked', outputCol='Embarked_i')

indexers = [sex_indexer, embarked_indexer]

pipeline = Pipeline(stages=indexers)

data = pipeline.fit(data).transform(data)

data.show(5)

+--------+------+------+----+-----+-----+-------+--------+-----+----------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Sex_i|Embarked_i|
+--------+------+------+----+-----+-----+-------+--------+-----+----------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|  0.0|       0.0|
|       1|     1|female|38.0|    1|    0|71.2833|       C|  1.0|       1.0|
|       1|     3|female|26.0|    0|    0|  7.925|       S|  1.0|       0.0|
|       1|     1|female|35.0|    1|    0|   53.1|       S|  1.0|       0.0|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|  0.0|       0.0|
+--------+------+------+----+-----+-----+-------+--------+-----+----------+
only showing top 5 rows



In [10]:
data= data.drop(*['Sex', 'Embarked'])

data.show(5)

+--------+------+----+-----+-----+-------+-----+----------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Sex_i|Embarked_i|
+--------+------+----+-----+-----+-------+-----+----------+
|       0|     3|22.0|    1|    0|   7.25|  0.0|       0.0|
|       1|     1|38.0|    1|    0|71.2833|  1.0|       1.0|
|       1|     3|26.0|    0|    0|  7.925|  1.0|       0.0|
|       1|     1|35.0|    1|    0|   53.1|  1.0|       0.0|
|       0|     3|35.0|    0|    0|   8.05|  0.0|       0.0|
+--------+------+----+-----+-----+-------+-----+----------+
only showing top 5 rows



In [11]:
data.describe().show()

+-------+------------------+------------------+-----------------+------------------+-------------------+------------------+------------------+------------------+
|summary|          Survived|            Pclass|              Age|             SibSp|              Parch|              Fare|             Sex_i|        Embarked_i|
+-------+------------------+------------------+-----------------+------------------+-------------------+------------------+------------------+------------------+
|  count|               712|               712|              712|               712|                712|               712|               712|               712|
|   mean|0.4044943820224719| 2.240168539325843|29.64209269662921|0.5140449438202247|0.43258426966292135| 34.56725140449432|0.3637640449438202|0.2612359550561798|
| stddev|0.4911389472541192|0.8368543166903446|14.49293290032352|0.9306921267673427| 0.8541814457454133|52.938648174710906|0.4814200635414358|0.5215606143930027|
|    min|                 0|

In [12]:
assembler = VectorAssembler(
    inputCols=["Pclass", "Age", "SibSp", "Parch", "Fare", "Sex_i", "Embarked_i"],
    outputCol="features")

data = assembler.transform(data)
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
pipeline = Pipeline(stages=[scaler])
scalerModel = pipeline.fit(data)
scaledData = scalerModel.transform(data)

train_data, test_data = scaledData.randomSplit([0.85, 0.15], seed=53)

In [13]:
scaledData.show(5)

+--------+------+----+-----+-----+-------+-----+----------+--------------------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Sex_i|Embarked_i|            features|     scaled_features|
+--------+------+----+-----+-----+-------+-----+----------+--------------------+--------------------+
|       0|     3|22.0|    1|    0|   7.25|  0.0|       0.0|[3.0,22.0,1.0,0.0...|[1.0,0.2711736617...|
|       1|     1|38.0|    1|    0|71.2833|  1.0|       1.0|[1.0,38.0,1.0,0.0...|[0.0,0.4722292033...|
|       1|     3|26.0|    0|    0|  7.925|  1.0|       0.0|[3.0,26.0,0.0,0.0...|[1.0,0.3214375471...|
|       1|     1|35.0|    1|    0|   53.1|  1.0|       0.0|[1.0,35.0,1.0,0.0...|[0.0,0.4345312892...|
|       0|     3|35.0|    0|    0|   8.05|  0.0|       0.0|(7,[0,1,4],[3.0,3...|(7,[0,1,4],[1.0,0...|
+--------+------+----+-----+-----+-------+-----+----------+--------------------+--------------------+
only showing top 5 rows



## Random Forest

In [14]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="Survived",
featuresCol="scaled_features"
, numTrees=30)
model = rf.fit(train_data)

predictions = model.transform(test_data)

predictions.show(3)

+--------+------+----+-----+-----+-------+-----+----------+--------------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Sex_i|Embarked_i|            features|     scaled_features|       rawPrediction|         probability|prediction|
+--------+------+----+-----+-----+-------+-----+----------+--------------------+--------------------+--------------------+--------------------+----------+
|       0|     1| 2.0|    1|    2| 151.55|  1.0|       0.0|[1.0,2.0,1.0,2.0,...|[0.0,0.0198542347...|[1.20425030619583...|[0.04014167687319...|       1.0|
|       0|     1|19.0|    3|    2|  263.0|  0.0|       0.0|[1.0,19.0,3.0,2.0...|[0.0,0.2334757476...|[17.0162930517742...|[0.56720976839247...|       0.0|
|       0|     1|28.0|    1|    0|82.1708|  0.0|       1.0|[1.0,28.0,1.0,0.0...|[0.0,0.3465694898...|[14.9721235505027...|[0.49907078501675...|       1.0|
+--------+------+----+-----+-----+-------+-----+----------+-----------

In [15]:
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy_rf = evaluator.evaluate(predictions)

print("Accuracy:", accuracy_rf)

Accuracy: 0.7850467289719626


## Árbol de Decisión

In [16]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="Survived",
featuresCol="scaled_features")
model = dt.fit(train_data)

In [17]:
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy_dt = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy_dt}")

Test Accuracy: 0.7757009345794392


## Naive Bayes

In [18]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(labelCol="Survived",
featuresCol="scaled_features")
nbmodel = nb.fit(train_data)

In [19]:
predictions = nbmodel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy_nb = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy_nb}")

Test Accuracy: 0.7850467289719626


## Regresión Logística

In [20]:
from pyspark.ml.classification import LogisticRegression

logistic_regression = LogisticRegression(featuresCol="scaled_features"
,labelCol="Survived")
model = logistic_regression.fit(train_data)

In [21]:
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy_lr = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy_lr}")

Test Accuracy: 0.7850467289719626


## Gradient-boosted tree

In [22]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="Survived"
,featuresCol="scaled_features"
, maxIter=10)
model = gbt.fit(train_data)

In [23]:
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy_gb = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy_gb}")

Test Accuracy: 0.7663551401869159


## Support Vector Machine

In [24]:
from pyspark.ml.classification import LinearSVC

lsvc = LinearSVC(labelCol="Survived"
,featuresCol="scaled_features"
,maxIter=10)
lsvcModel = lsvc.fit(train_data)

In [25]:
predictions = lsvcModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy_sv = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy_sv}")

Test Accuracy: 0.7850467289719626


## PerceptronClassifier

In [26]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

mlpc=MultilayerPerceptronClassifier( featuresCol="scaled_features",labelCol="Survived" ,layers = [7,16,2], maxIter=1000,blockSize=8,seed=7,solver="gd")

In [27]:
ann = mlpc.fit(train_data)

In [28]:
pred = ann.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol='prediction',metricName='accuracy')
ann_accuracy = evaluator.evaluate(pred)
print(f"Test Accuracy: {ann_accuracy}")

Test Accuracy: 0.794392523364486
